## Zentralitätsmetriken

PageRank: Identifiziert die Einflussreichsten Knoten basierend darauf, wie viele Verbindungen (Kanten) auf sie zeigen und wie einflussreich die Knoten sind, die auf sie zeigen. <br>
Betweeness Centralitiy: Misst, wie oft ein Knoten auf dem kürzesten Pfad zwischen anderen Knotenpaaren liegt. Hohe Betweenness-Werte weisen auf Knoten hin, die als wichtige Vermittler innerhalb des Netzwerks fungieren 

#### Cypher Pagerank Query

In [13]:
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", ""))

pagerank_query = """
CALL gds.pageRank.stream('graphAcc20')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).accountNumber AS account, score
ORDER BY score DESC
"""
pagerank_results = graph.run(pagerank_query).data()

for result in pagerank_results:
    print(result)

{'account': 'A1', 'score': 1.1700328390646446}
{'account': 'A4', 'score': 1.1700328390646446}
{'account': 'A2', 'score': 0.828981500254398}
{'account': 'A3', 'score': 0.828981500254398}
{'account': 'A5', 'score': 0.8081736659393426}


Output Deutung: <br>
Zeigt die "Wichtigkeit" oder den Einfluss jedes Knotens (repräsentiert durch Personen) in Ihrem Netzwerk, basierend auf der PageRank-Metrik. <br>

#### Übergabe an GPT

In [14]:
import openai

class OpenAIWrapper:
    def __init__(self, api_key):
        self.api_key = api_key
        openai.api_key = self.api_key

    def generate_chat_response(self, prompt, model="gpt-3.5-turbo", temperature=0.7, max_tokens=150):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].message['content'].strip() 
        except Exception as e:
            return f"An error occurred: {str(e)}"

In [17]:
openai_wrapper = OpenAIWrapper(api_key='')

prompt_intro = "Based on the PageRank analysis of the network, the nodes are ranked according to their influence within the network. The PageRank score indicates the likelihood of arriving at a node through random walks. Here are the top-ranked nodes based on their PageRank scores:\n\n"

prompt_body = "\n".join([f"- {res['account']} with a PageRank score of {res['score']:.2f}" for res in pagerank_results[:10]])  # Top 10 für den Prompt

prompt_question = "\n\nWhat insights can be drawn from these PageRank scores regarding the structure and influence dynamics within the network? Additionally, what further analyses could provide deeper understanding of the network's characteristics?"

complete_prompt = prompt_intro + prompt_body + prompt_question

print(complete_prompt)

response = openai_wrapper.generate_chat_response(complete_prompt)
print(f"GPT-3's Insights: {response}")

Based on the PageRank analysis of the network, the nodes are ranked according to their influence within the network. The PageRank score indicates the likelihood of arriving at a node through random walks. Here are the top-ranked nodes based on their PageRank scores:

- A1 with a PageRank score of 1.17
- A4 with a PageRank score of 1.17
- A2 with a PageRank score of 0.83
- A3 with a PageRank score of 0.83
- A5 with a PageRank score of 0.81

What insights can be drawn from these PageRank scores regarding the structure and influence dynamics within the network? Additionally, what further analyses could provide deeper understanding of the network's characteristics?
GPT-3's Insights: An error occurred: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You ca

#### Cypher Betweenness Query

In [18]:
betweenness_query = """
CALL gds.betweenness.stream('graphAcc20')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).accountNumber AS account, score
ORDER BY score DESC
"""

betweenness_results = graph.run(betweenness_query).data()

for result in betweenness_results:
    print(result)

{'account': 'A1', 'score': 1.5}
{'account': 'A4', 'score': 1.5}
{'account': 'A2', 'score': 0.5}
{'account': 'A3', 'score': 0.5}
{'account': 'A5', 'score': 0.0}


Output Deutung: <br>
James Becker und Oliver Schulz zeigen sehr hohe Betweenness-Werte, was darauf hindeutet, dass sie wichtige Verbindungspunkte im Netzwerk sind <br>
Max Mustermann, Jane Doe, und John Smith haben eine Betweenness Centrality von 0, was bedeutet, dass sie nicht auf kürzesten Pfaden zwischen anderen Knoten liegen. 

#### Übergabe an GPT

In [19]:
prompt_intro = "Based on the Betweenness Centrality analysis of the network, the nodes are ranked according to their roles as intermediaries in the communication paths across the network. The Betweenness Centrality score indicates how often a node acts as a bridge along the shortest path between two other nodes. Here are the nodes with the highest Betweenness Centrality scores:\n\n"

prompt_body = "\n".join([f"- {res['account']} with a Betweenness Centrality score of {res['score']:.2f}" for res in betweenness_results[:10]])  #  Top 10 für den Prompt

prompt_question = "\n\nWhat can be inferred about the network's structure and the importance of these nodes from these Betweenness Centrality scores? What implications do these scores have for the flow of information or influence within the network?"

complete_prompt = prompt_intro + prompt_body + prompt_question

print(complete_prompt)

response = openai_wrapper.generate_chat_response(complete_prompt)
print(f"GPT-3's Insights: {response}")

Based on the Betweenness Centrality analysis of the network, the nodes are ranked according to their roles as intermediaries in the communication paths across the network. The Betweenness Centrality score indicates how often a node acts as a bridge along the shortest path between two other nodes. Here are the nodes with the highest Betweenness Centrality scores:

- A1 with a Betweenness Centrality score of 1.50
- A4 with a Betweenness Centrality score of 1.50
- A2 with a Betweenness Centrality score of 0.50
- A3 with a Betweenness Centrality score of 0.50
- A5 with a Betweenness Centrality score of 0.00

What can be inferred about the network's structure and the importance of these nodes from these Betweenness Centrality scores? What implications do these scores have for the flow of information or influence within the network?
GPT-3's Insights: An error occurred: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authoriza